<font size=+3 color="#3B3534"><center><b>Housing in London 🏘️💷💂</b></center></font>

<img src="https://images.unsplash.com/photo-1501683567677-568cfd15361c?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1414&q=80" width="750">
<center>Photo by Nigel Tadyanehondo on Unsplash</center>

<br>

# Introduction

Hello readers! The topic of this notebook is the housing market of London in the years between 1995 and 2020. We are going to perform exploratory data analysis with the goal of discovering new information and answering the following questions:

- How has the housing market (average house price and number of houses sold) changed over the years in different boroughs of London? How does it compare to England?
- What factors affected it the most?

<br>

**Table of Contents**

1. [Introduction](#Introduction)
2. [Libraries](#Libraries)
3. [Meet the Datasets](#Datasets)
4. [London Housing over the Years](#Housing-over-the-Years)
5. [Factors Affecting Housing](#Factors-Affecting-Housing)
6. [Conclusions](#Conclusions)
7. [Future Developement](#Future-Developement)

# Libraries

We start by importing the necessary libraries and setting some parameters for the whole notebook (e.g. the display format for the pandas library).

In [1]:
import numpy as np

import pandas as pd
from pandas.plotting import scatter_matrix

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.float_format = '{:,.2f}'.format
# pd.set_option('precision', 2)
font_size = 17

In [ ]:
df_m = pd.read_csv('../input/housing_in_london_monthly_variables.csv', parse_dates = ['date'])
df_y = pd.read_csv('../input/housing_in_london_yearly_variables.csv', parse_dates = ['date'])

In [ ]:
df = df_m
intended_df_size_in_MB = 256
factor = intended_df_size_in_MB*(2**20)/(df_m.memory_usage(index=True).sum()+df_y.memory_usage(index=True).sum())
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
df_m = df

df = df_y
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
df_y = df
    
df_m.info()
df_y.info()

# Datasets

We are going to use the [Housing in London](https://www.kaggle.com/justinas/housing-in-london) dataset which is provided by [Justinas Cirtautas](https://www.kaggle.com/justinas). It contains two csv files ('**housing_in_london_monthly_variables.csv**' and '**housing_in_london_yearly_variables.csv'**) with a lot of relevant information such as the monthly average house prices, yearly mean and median salary for residents of each area, etc.

The data is split by areas of London called boroughs, but some of the instances correspond to other UK regions (like North East, West Midlands, etc.).

Additionally, we are going to use a third dataset for plotting maps ([London Borough and Ward Boundaries up to 2014](https://www.kaggle.com/csobral/london-borough-and-ward-boundaries-up-to-2014/notebooks), provided by [Cayo Costa Sobral](https://www.kaggle.com/csobral)).

# Housing over the Years

We are going to focus on the **average price** and the **number of houses sold** in each area. 

For this task, we need two datasets:
- '**housing_in_london_monthly_variables**' which contains monthly information about the London boroughs and the other UK regions, and
- '**London_Borough_Excluding_MHW.shp**' for plotting maps.

## Data Preprocessing

Let's import the first dataset and take a quick look at the data:

In [3]:
#df_m = pd.read_csv('../input/housing_in_london_monthly_variables.csv', parse_dates = ['date'])

print ('This dataset contains {} rows and {} columns.'.format(df_m.shape[0], df_m.shape[1]))
df_m.head()

This dataset contains 13549 rows and 7 columns.


,date,area,average_price,code,houses_sold,no_of_crimes,borough_flag
0,1995-01-01,city of london,91449,E09000001,17.00,NaN,1
1,1995-02-01,city of london,82203,E09000001,7.00,NaN,1
2,1995-03-01,city of london,79121,E09000001,14.00,NaN,1
3,1995-04-01,city of london,77101,E09000001,7.00,NaN,1
4,1995-05-01,city of london,84409,E09000001,10.00,NaN,1


There are 7 attributes in total. An instance represents a London borough if 'borough_flag' equals 1. The meaning of the rest of the attributes can be easily inferred from their name.

The method `info()` can give us valuable information such as the type and the number of missing values in each attribute:

In [4]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           13549 non-null  datetime64[ns]
 1   area           13549 non-null  object        
 2   average_price  13549 non-null  int64         
 3   code           13549 non-null  object        
 4   houses_sold    13455 non-null  float64       
 5   no_of_crimes   7439 non-null   float64       
 6   borough_flag   13549 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 741.1+ KB


In [5]:
null_df_m = df_m.isnull().sum().sort_values(ascending = False)
percent = (df_m.isnull().sum()/df_m.isnull().count()).sort_values(ascending = False)*100

null_df_m = pd.concat([null_df_m, percent], axis = 1, keys = ['Counts', '% Missing'])
print ('Missing: ')
null_df_m.head()

Missing: 


,Counts,% Missing
no_of_crimes,6110,45.10
houses_sold,94,0.69
date,0,0.00
area,0,0.00
average_price,0,0.00


Two attributes, 'houses_sold' and 'no_of_crimes', have missing values. We will drop the whole 'no_of_crimes' attribute since almost 50% of its instances have NaN values. 

It's easier to replace missing values in the 'houses_sold' attribute as only a small portion is missing. We will use the mean value for the same area for all years in each case. Of course, someone could argue that this number would change over the years, but we could assume that the final results won't be affected due to small number of values being changed.

In [6]:
df_m.drop('no_of_crimes', axis = 1, inplace = True)   # drop the 'no_of_crimes column

df_m['houses_sold'].fillna(df_m.groupby('area')['houses_sold'].transform('mean'), inplace = True) # fill NaN values with the mean of that particular area

The 'date' attribute is of datatime type, hence we can extract the corresponding year of each instance (month and day are not important):

In [7]:
df_m['year'] = df_m['date'].dt.year
df_m.iloc[[0, -1]]

,date,area,average_price,code,houses_sold,borough_flag,year
0,1995-01-01,city of london,91449,E09000001,17.00,1,1995
13548,2020-01-01,england,247355,E92000001,"77,182.73",0,2020


The dataset contains samples from January 1995 to January 2020. We will discard all samples from 2020 since the year is not complete:

In [8]:
df_m = df_m[df_m['year'] < 2020]
df_m['year'].max()

2019

## Data Exploration

According to [Wikipedia](https://en.wikipedia.org/wiki/List_of_London_boroughs), there are 33 boroughs in London (32 + the City of London). Is that the case in our dataset?

In [9]:
lnd_boroughs = df_m[df_m['borough_flag'] == 1]['area'].unique()
len(lnd_boroughs)

33

It is! How many and which regions are outside of London?

In [10]:
df_m[df_m['borough_flag'] == 0]['area'].nunique()

12

In [11]:
df_m[df_m['borough_flag'] == 0]['area'].unique()

array(['south east', 'inner london', 'outer london', 'north east',
       'north west', 'yorks and the humber', 'east midlands',
       'west midlands', 'east of england', 'london', 'south west',
       'england'], dtype=object)

We can manually isolate the ones that correspond to a region in England: 

In [12]:
eng_regions = ['south west', 'south east', 'east of england', 'west midlands', 'east midlands', 'yorks and the humber', 'north west', 'north east']

### London VS England

We can now split the dataset into two: one for boroughs in London and one for the other regions of England:

In [13]:
lnd = df_m[df_m['area'].isin(lnd_boroughs)]
eng = df_m[df_m['area'].isin(eng_regions)]

#### Average Price

The `groupby()` method allows us to calculate the mean 'average price' for each 'date':

In [14]:
lnd_pr = lnd.groupby('date')['average_price'].mean()
eng_pr = eng.groupby('date')['average_price'].mean()

- In overall, the averace price follows an **upward trend** during the studied time frame, with London always having a higher average price. 
- This upward trend was disrupted by the the **[Great Recession](https://en.wikipedia.org/wiki/Great_Recession)** (2007-209), hence the significant **decline starting from 2007**. London apparently **recovered quickly** and the average price increased rapidly until 2016. **England** followed a similar behaviour but with a **moderate rise** in the same period.
- Arguably, the **[Brexit referendum](https://en.wikipedia.org/wiki/2016_United_Kingdom_European_Union_membership_referendum)** had an impact in London housing since the average price hit a **plateau after 2016**. It seems the the rest of England wasn't affected as much from the referendum.

<br>

<font color="#842A16" size=+0><b>Gifs</b></font> 

I really like adding another dimension in my graphs, by making **gifs**. I recently came across the [**gif**](https://github.com/maxhumber/gif) library by Max Humber, which allows us to make gifs simply by building a bunch of "frames" with a standard for loop.

The steps are the following:

- Create a 'plot' function which plots the average price for both London (df_lnd) and England (df_eng) until the specified date (third argument). It doesn't have to return something.
- Decorate the plot function with @gif.frame, 
- Use a for loop to create a frame for each month, and append the result to a list (called 'frames' in our case). Finally,
- Use the `save()` method from the gif library to save it.

In [15]:
# frames = []
# for months in pd.date_range(start = lnd_pr.index[0], end = lnd_pr.index[-1], freq = '3MS'): # 3MS --> every three months
#     frame = plot(lnd_pr, eng_pr, months)
#     frames.append(frame)
    
# gif.save(frames, 'Price-Lnd_vs_Eng.gif', duration = 1, unit = 's', between = 'startend')

We can insert our gif in the notebook:

In [16]:
# HTML('<img src="./Price-Lnd_vs_Eng.gif" />')

<br>

The average price in each borough fluctuates through time. However, we can calculate its mean which can give us a rough indication of how expensive each area is.

In [17]:
lnd_b_prices = lnd.groupby('area')['average_price'].mean()
lnd_top10_pr = lnd_b_prices.sort_values(ascending = False).to_frame()

print ('\nThe 10 most expensive boroughs in London are:')
lnd_top10_pr.head(10)


The 10 most expensive boroughs in London are:


,average_price
area,
kensington and chelsea,"740,606.68"
westminster,"542,282.06"
camden,"466,358.25"
hammersmith and fulham,"442,773.38"
city of london,"422,771.36"
richmond upon thames,"381,640.97"
islington,"364,160.28"
wandsworth,"341,411.03"
barnet,"304,272.12"


In [18]:
lnd_top10_pr.head(10).sort_values(by = 'average_price', ascending = True)

,average_price
area,
haringey,"292,017.11"
barnet,"304,272.12"
wandsworth,"341,411.03"
islington,"364,160.28"
richmond upon thames,"381,640.97"
city of london,"422,771.36"
hammersmith and fulham,"442,773.38"
camden,"466,358.25"
westminster,"542,282.06"


We focus on the 5 most expensive boroughs and see the evolution of the average price through time:

In [19]:
top5_indeces = lnd_top10_pr.head().index
colors = ['#e74c3c', '#3498db', '#95a5a6', '#34495e', '#2ecc71']


for index, i in enumerate(top5_indeces):
    df_ = lnd[lnd['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()

- **Kensington and Chelsea** is the **most expensive borough** in London between 1995 and 2019. We can read in [wikipedia](https://en.wikipedia.org/wiki/Royal_Borough_of_Kensington_and_Chelsea) that it holds royal status and it includes many affluent areas, hence the increased price.
- **Westministe**r, which is adjacent to Kensighton, comes **second**. Again this is not a surpise since it is the location of the national government and includes many sites commonly associated with London (e.g. the Buckingham Palace, the Houses of Parliament, 10 Downing Street, and Trafalgar Square) ([Source](https://en.wikipedia.org/wiki/City_of_Westminster)).

We can perform a similar analysis for England:

In [20]:
eng_prices = eng.groupby('area')['average_price'].mean()
eng_top3_pr = eng_prices.sort_values(ascending = False).to_frame()

print('The top 3 most expensive regions in England are:')
eng_top3_pr.head(3)

The top 3 most expensive regions in England are:


,average_price
area,
south east,"195,653.01"
east of england,"170,796.17"
south west,"163,050.03"


In [21]:
top3_indeces = eng_top3_pr.head(3).index
colors = ['darkorange', '#8EB8E5', 'forestgreen', ]

for index, i in enumerate(top3_indeces):
    df_ = eng[eng['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()

The three **most expensive English regions** are all located in the **South of England** (two of them border London). It would be interesting to plot the same graph but with the cheapest borough of London (which we can find is Barking and Dagenham):

In [22]:


for index, i in enumerate(top3_indeces):
    df_ = eng[eng['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()

lnd_bng_pr = lnd[lnd['area'] == 'barking and dagenham'].groupby('date')['average_price'].mean()

It is evident that even the cheapest borough in London is **comparable** to the most expensive regions of England!

### Houses Sold

Let's explore the 'houses_sold' attribute for London.

In [23]:
lnd_houses = lnd.groupby('date')['houses_sold'].sum()

- The number of houses sold in London **dropped sharply** during the **financial recession** and it seems that it **hasn't completely recovered** since,  
- Interestingly, there is **spike in March 2016** which at first sight looks like a artefact. That is not the case, because people were actually trying to avoid a **new legislation** that came into effect in April 2016 and imposed an increase in the tax bill on buying a second home (Credits to [Marcel Fellipe](https://www.kaggle.com/marcelfellipe/housing-in-london-exploratory-data-analysis) for finding this). 
- Again, the **referendum** resulted in a **downward trend** starting from July 2016.

<br>

The boroughs with the highest number of sold houses between 1995 and 2019 are:

In [24]:
lnd_b_houses = lnd.groupby('area')['houses_sold'].sum()
lnd_top5_h = lnd_b_houses.sort_values(ascending = False).to_frame()
lnd_top5_h.head(5)

,houses_sold
area,
wandsworth,"164,419.06"
bromley,"149,012.71"
croydon,"146,393.98"
barnet,"134,094.98"
lambeth,"118,347.49"


To better understand these numbers we need more information such as the size of the borough, its population, crime rates etc. Some of these information are available in the other dataset ('housing_in_london_yearly_variables').

## Map of London

We are going to use the **second dataset** which contains geospatial information about London (specifically, the 'geometry' attribute). This [notebook](https://www.kaggle.com/justinas/house-prices-in-london) helped me a lot for the analysis.

We start by merging the two datasets:

Remember that London has 33 boroughs, same as the lnd_tot dataset. Let's check that the london_map dataset has the same names:

It does, so we are good to go.

The merged dataset has a new type, **geodataframe**, which is what we need for plotting the map.

- We can now see Kensighton and Westminister as the two dark boroughs in the first map. We can roughly say that boroughs **closer** to these two areas are **more expensive**.
- As we hinted earlier, the number of houses sold is probably affected by many factors with one of them being the **total area** of each borough. We can definetely see that larger boroughs have more houses sold.

---

# Factors Affecting Housing

## Preparing the Final Dataset

For this question we need to import the third and final dataset ('**housing_in_london_yearly_variables**') which we are going to merge with the monthly dataset.

In [25]:
#df_y = pd.read_csv('../input/housing_in_london_yearly_variables.csv', parse_dates = ['date'])
df_y = df_y[df_y['area'].isin(lnd_boroughs)] # select only London boroughs

print ('This dataset contains {} rows and {} columns.'.format(df_y.shape[0], df_y.shape[1]))
df_y.head()

This dataset contains 693 rows and 12 columns.


,code,area,date,median_salary,life_satisfaction,mean_salary,recycling_pct,population_size,number_of_jobs,area_size,no_of_houses,borough_flag
0,E09000001,city of london,1999-12-01,"33,020.00",NaN,48922,0,"6,581.00",NaN,NaN,NaN,1
1,E09000002,barking and dagenham,1999-12-01,"21,480.00",NaN,23620,3,"162,444.00",NaN,NaN,NaN,1
2,E09000003,barnet,1999-12-01,"19,568.00",NaN,23128,8,"313,469.00",NaN,NaN,NaN,1
3,E09000004,bexley,1999-12-01,"18,621.00",NaN,21386,18,"217,458.00",NaN,NaN,NaN,1
4,E09000005,brent,1999-12-01,"18,532.00",NaN,20911,6,"260,317.00",NaN,NaN,NaN,1


This dataset has more information for each borough and is ideal for answering the second question. Let's see how many missing values each attrbitutes has:

In [26]:
null_df_y = df_y.isnull().sum().sort_values(ascending = False)
percent = (df_y.isnull().sum()/df_y.isnull().count()).sort_values(ascending = False)*100

null_df = pd.concat([null_df_y, percent], axis = 1, keys = ['Counts', '%'])
null_df.head(10)

,Counts,%
life_satisfaction,437,63.06
area_size,99,14.29
no_of_houses,99,14.29
number_of_jobs,66,9.52
recycling_pct,33,4.76
population_size,33,4.76
median_salary,10,1.44
code,0,0.00
area,0,0.00
date,0,0.00


In [27]:
# import missingno as msno
# msno.matrix(df_y)

It seems that all new attributes have missing values. Pandas' `corr()` function, which we are going to use later, ignores a pairwise correlation if there is a NaN value in one of the observations, so we shouldn't worry about it for now.

'median_salary' is not currently numeric. If we try to convert it to numeric (with the `astype()` method), we'll get an error. That's because some instances contain symbols and need to be replaced first:

In [28]:
df_y[~df_y['mean_salary'].str.isnumeric()]['mean_salary'].value_counts()

mean_salary
#    7
Name: count, dtype: int64

In [29]:
df_y['mean_salary'] = df_y['mean_salary'].replace(['#'], np.NaN)
df_y['mean_salary'] = df_y['mean_salary'].astype(float)

In [30]:
df_y['year'] = df_y['date'].dt.year

print ('yearly_variables dataset')
print ('\tFirst date: ', df_y['year'].min())
print ('\tFinal date: ', df_y['year'].max())

yearly_variables dataset
	First date:  1999
	Final date:  2019


This dataset starts four years later compared to the monthly dataset. Therefore, we need to merge them so that there is no mismatch in the 'year' column:

The two datasets are ready to be merged:

## Correlations

- It doesn't come as a surpise that apart from the year, the average price in London has a **significant positive correlation** with the **median and mean salary** along with the **number of jobs** in a borough. 
- There is **small positive correlation** with **life satisfaction**. More data for this attribute would be useful.
- As we discussed earlier, the most affluent boroughs happen to be small (see map) which explains the **negative correlation** with the **size** of an area. 
- The average price is also affected by the **number of houses sold**, since the former goes up when the later decreases (people sell higher when less houses are sold?).

We can summarise these correlations (along with all pair-wise correlations) with the following heatmap:

We can see some obvious correlations such as median-mean salary, houses sold-no_of_houses, no_of_houses-population_size, etc.

Lastly, a linear relationship between attributes can be visualised with the `scatter_matrix` method:

# Conclusions

The main conclusions are the following:

- The average price in **London boroughs** is **higher** compared to the rest of England. It has been **affected** by major financial and political events (such the Recession and Brexit), but has **significantly increased** from 1995 to 2019 (by a factor of 5),

- **Affluent regions** such as Kensighton and Westiminister have the **highest average price**, 

- The number of houses sold **plummeted** after the recession, **streadily increased** until 2016 but then again **dropped** after the referendum, 

- As anyone could predict, the main factor that influences the average price is the **financial prosperity** of the corresponding borough. Higher salaries and more jobs result in higher prices.

# Future Developement

This notebook could be further improved by making some additions:

- Use maps to visualise how variables in the 'yearly_dataset' vary from borough to borough,
- Compare English regions with London (all boroughs) with respect to the other variables (maps could be useful),
- Use Folium for plotting interactive maps.
    
<br>
Please <font color="red" size=+0><b>upvote</b></font> if you liked this notebook! Thank you! 😉